In [6]:
!pip install pandas
import pandas as pd
df = pd.read_csv("linkedin_all_tech_public_jobs.csv")
df.head()


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


,title,url
0,Software Engineer | Makati,https://ph.linkedin.com/jobs/view/software-eng...
1,NaN,https://ph.linkedin.com/jobs/view/junior-front...
2,NaN,https://ph.linkedin.com/jobs/view/junior-front...
3,NaN,https://ph.linkedin.com/jobs/view/junior-full-...
4,NaN,https://ph.linkedin.com/jobs/view/junior-back-...


In [2]:
!pip install selenium
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time, random


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
def scrape_job_details(driver, job_links):
    results = []

    for i, link in enumerate(job_links, 1):
        job_data = {"url": link, "title": None, "company": None, "location": None, "description": None}
        print(f"[{i}/{len(job_links)}] Scraping: {link}")

        driver.get(link)
        time.sleep(random.uniform(2.5, 4.5))

        try:
            # --- Job title ---
            title_elem = driver.find_element(By.CSS_SELECTOR, "h1.topcard__title")
            job_data["title"] = title_elem.text.strip()
        except NoSuchElementException:
            pass

        try:
            # --- Company name ---
            company_elem = driver.find_element(By.CSS_SELECTOR, "a.topcard__org-name-link")
            job_data["company"] = company_elem.text.strip()
        except NoSuchElementException:
            pass

        try:
            # --- Location ---
            location_elem = driver.find_element(By.CSS_SELECTOR, "span.topcard__flavor--bullet")
            job_data["location"] = location_elem.text.strip()
        except NoSuchElementException:
            pass

        try:
            # --- Job description ---
            desc_elem = driver.find_element(By.CSS_SELECTOR, "div.show-more-less-html__markup")
            job_data["description"] = desc_elem.text.strip()
        except NoSuchElementException:
            pass
        
        results.append(job_data)
        time.sleep(random.uniform(1.5, 3.0))

    return results

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
!pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote_plus


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import tempfile
def make_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
        options.add_argument("--no-sandbox") # Required in Colab for headless mode
        options.add_argument("--disable-dev-shm-usage") # Required in Colab for headless mode
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    # Add user data directory argument with a temporary path to avoid "user data directory already in use" error
    options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")
    # Removed Service(ChromeDriverManager().install()) as it's not needed in Colab
    driver = webdriver.Chrome(options=options)
    return driver

In [7]:
job_links = list(df['url'])

In [8]:
#function that receives a list of job details and saves them to a CSV file
def save_to_csv(job_details, filename):
    df = pd.DataFrame(job_details)
    df.to_csv(filename, index=False)
    print(f"✅ Saved {len(job_details)} job details to {filename}.")

In [ ]:
driver = make_driver(headless=True)  # Set headless=False if you want to see the browser

try:
    all_jobs = []
    batch_size = 100
    start_index = 1600  # Starting point

    # Loop from index 1600 up to the last
    for start in range(start_index, len(job_links), batch_size):
        end = start + batch_size
        print(f"🔍 Scraping jobs {start + 1} to {min(end, len(job_links))}...")

        # Scrape this batch
        jobs_batch = scrape_job_details(driver, job_links[start:end])
        all_jobs.extend(jobs_batch)

        # Calculate part number relative to the starting index
        part_num = (start - start_index) // batch_size + 1

        # Save to CSV every batch
        save_to_csv(jobs_batch, f"scraped_job_details_part_{part_num}.csv")
        print(f"💾 Saved batch {part_num} with {len(jobs_batch)} jobs.")

    print(f"✅ Successfully scraped and saved details for {len(all_jobs)} total jobs (from index {start_index}).")

finally:
    driver.quit()  # Always close the driver

🔍 Scraping jobs 201 to 300...
[1/100] Scraping: https://ph.linkedin.com/jobs/view/engineer-ai-ml-full-stack-developer-lb-09162025-ftefsd-at-rippedboxstation-4301909394?position=21&pageNum=0&refId=TTt60Vw2D91LnO5M3VAXGg%3D%3D&trackingId=XMUs%2FnimXRga7XIjh4DTXg%3D%3D
[2/100] Scraping: https://ph.linkedin.com/jobs/view/investor-relations-and-corporate-planning-data-scientist-at-bdo-unibank-4309477437?position=22&pageNum=0&refId=TTt60Vw2D91LnO5M3VAXGg%3D%3D&trackingId=PE2kXuJR4Zj%2Fkf9LEkTDkw%3D%3D
[3/100] Scraping: https://ph.linkedin.com/jobs/view/data-scientist-at-teksynap-4301597837?position=23&pageNum=0&refId=TTt60Vw2D91LnO5M3VAXGg%3D%3D&trackingId=f3lfy8l61tPhMdLQvCo%2BHQ%3D%3D
[4/100] Scraping: https://ph.linkedin.com/jobs/view/ai-ml-engineer-at-emma-of-torre-ai-4318177391?position=24&pageNum=0&refId=TTt60Vw2D91LnO5M3VAXGg%3D%3D&trackingId=%2FdYRZcnvcAAD54tvLpCMww%3D%3D
[5/100] Scraping: https://ph.linkedin.com/jobs/view/join-our-amazing-tech-team-get-26k-salary-%2B-35k-incentives-

KeyboardInterrupt: 

In [12]:
#combine all saved CSV files into a single dataframe
df = pd.read_csv("scraped_job_details_part_1.csv")
df2 = pd.read_csv("scraped_job_details_part_2.csv")
df3 = pd.read_csv("scraped_job_details_part_3.csv")
df4 = pd.read_csv("scraped_job_details_part_4.csv")
#up to 14
df5 = pd.read_csv("scraped_job_details_part_5.csv")
df6 = pd.read_csv("scraped_job_details_part_6.csv")
df7 = pd.read_csv("scraped_job_details_part_7.csv")
df8 = pd.read_csv("scraped_job_details_part_8.csv")
df9 = pd.read_csv("scraped_job_details_part_9.csv")
df10 = pd.read_csv("scraped_job_details_part_10.csv")
df11 = pd.read_csv("scraped_job_details_part_11.csv")
df12 = pd.read_csv("scraped_job_details_part_12.csv")
df13 = pd.read_csv("scraped_job_details_part_13.csv")
df14 = pd.read_csv("scraped_job_details_part_14.csv")
df15 = pd.read_csv("scraped_job_details_100.csv")
df16 = pd.read_csv("scraped_job_details_200.csv")

df_combined = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16], ignore_index=True)


In [14]:
df_combined.info()
df_combined.to_csv("linkedin_scraped_job_details_1600.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          1600 non-null   object
 1   title        1597 non-null   object
 2   company      1597 non-null   object
 3   location     1597 non-null   object
 4   description  1246 non-null   object
dtypes: object(5)
memory usage: 62.6+ KB
